# Neuron set examples

In [3]:
import obi_one as obi
import os

### __Initialization:__ Loading a circuit

In [4]:
circuit = obi.Circuit(name="ToyCircuit-S1-6k", path="/Users/pokorny/Data/Circuits/ToyCircuit-S1-6k/circuit_config.json")
print(f"Circuit '{circuit}' with {circuit.sonata_circuit.nodes.size} neurons and {circuit.sonata_circuit.edges.size} synapses")
print(f"Default node population: '{circuit.default_population_name}'")

Circuit 'ToyCircuit-S1-6k' with 5924 neurons and 568717 synapses
Default node population: 'All'


### __Example 1:__ Adding node set dict to an existing SONATA circuit object + writing new node set .json file

In [5]:
# Get SONATA circuit object
c = circuit.sonata_circuit
print("..." + str(c.node_sets.content)[-25:])

# Adding a node set to the circuit
obi.NeuronSet.add_node_set_to_circuit(c, {"Layer23": {"layer": [2, 3]}})
print("..." + str(c.node_sets.content)[-55:])

# Adding a node set with an exising name => NOT POSSIBLE
# obi.NeuronSet.add_node_set_to_circuit(c, {"Layer23": {"layer": [2, 3]}})  # AssertionError: Node set 'Layer23' already exists!

# Update/overwrite an existing node set
obi.NeuronSet.add_node_set_to_circuit(c, {"Layer23": ["Layer2", "Layer3"]}, overwrite_if_exists=True)  # Update/overwrite
print("..." + str(c.node_sets.content)[-58:])

# Adding multiple node sets
obi.NeuronSet.add_node_set_to_circuit(c, {"Layer45": ["Layer4", "Layer5"], "Layer56": ["Layer5", "Layer6"]})
print("..." + str(c.node_sets.content)[-124:])

# Add node set from NeuronSet object, resolved in circuit's default node population
neuron_set = obi.CombinedNeuronSet(node_sets=("Layer1", "Layer2", "Layer3"))
obi.NeuronSet.add_node_set_to_circuit(c, {"Layer123": neuron_set.get_node_set_definition(circuit, circuit.default_population_name)})
print("..." + str(c.node_sets.content)[-168:])

# Adding a node sets based on previously added node sets
obi.NeuronSet.add_node_set_to_circuit(c, {"AllLayers": ["Layer123", "Layer4", "Layer56"]})
print("..." + str(c.node_sets.content)[-216:])

# Write new circuit's node set file
obi.NeuronSet.write_circuit_node_set_file(c, output_path="./", file_name="new_node_sets.json", overwrite_if_exists=True)

..., 'Layer6': {'layer': 6}}
..., 'Layer6': {'layer': 6}, 'Layer23': {'layer': [2, 3]}}
..., 'Layer6': {'layer': 6}, 'Layer23': ['Layer2', 'Layer3']}
..., 'Layer6': {'layer': 6}, 'Layer23': ['Layer2', 'Layer3'], 'Layer45': ['Layer4', 'Layer5'], 'Layer56': ['Layer5', 'Layer6']}
..., 'Layer6': {'layer': 6}, 'Layer23': ['Layer2', 'Layer3'], 'Layer45': ['Layer4', 'Layer5'], 'Layer56': ['Layer5', 'Layer6'], 'Layer123': ['Layer1', 'Layer2', 'Layer3']}
..., 'Layer6': {'layer': 6}, 'Layer23': ['Layer2', 'Layer3'], 'Layer45': ['Layer4', 'Layer5'], 'Layer56': ['Layer5', 'Layer6'], 'Layer123': ['Layer1', 'Layer2', 'Layer3'], 'AllLayers': ['Layer123', 'Layer4', 'Layer56']}


### __Example 2:__ Use of different NeuronSet types

<u>Important</u>: In general, the validity of neuron set definitions is not checked during initialization, but when resolved within a specific circuit's node population

#### (a) __ExistingNeuronSet__, wrapper for an existing node set
<u>Note:</u> This neuron set does not resolve to a dict, since the underlying node set is already existing by definition

In [6]:
neuron_set = obi.ExistingNeuronSet(node_set="Layer1", random_sample=None)
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

ExistingNeuronSet resolved in population 'All' of circuit 'ToyCircuit-S1-6k':
> Neuron IDs (70): [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69]
> Node set dict: None


#### (b) __ExistingNeuronSet__, with random sub-sampling
<u>Note</u>: `random_sample` can be an absolute number or fraction

<u>Note 2</u>: Random sub-sampling will enforce resolving into a new node set

In [7]:
neuron_set = obi.ExistingNeuronSet(node_set="Layer1", random_sample=10, random_seed=1)
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

ExistingNeuronSet resolved in population 'All' of circuit 'ToyCircuit-S1-6k':
> Neuron IDs (10): [10 24 33 35 47 59 61 63 66 67]
> Node set dict: {'population': 'All', 'node_id': [10, 24, 33, 35, 47, 59, 61, 63, 66, 67]}


#### (c) __CombinedNeuronSet__, based on combining existing (named) node sets

In [8]:
neuron_set = obi.CombinedNeuronSet(circuit=circuit, population="All", node_sets=("Layer1", "Layer2", "Layer3"), random_sample=None)
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

CombinedNeuronSet resolved in population 'All' of circuit 'ToyCircuit-S1-6k':
> Neuron IDs (1659): [   0    1    2 ... 1656 1657 1658]
> Node set dict: ['Layer1', 'Layer2', 'Layer3']


#### (d) __CombinedNeuronSet__, based on combining existing (named) node sets, with random sub-sampling
<u>Note</u>: `random_sample` can be an absolute number or fraction

In [9]:
neuron_set = obi.CombinedNeuronSet(node_sets=("Layer1", "Layer2", "Layer3"), random_sample=10, random_seed=0)
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

CombinedNeuronSet resolved in population 'All' of circuit 'ToyCircuit-S1-6k':
> Neuron IDs (10): [ 115  145  215  355  378  568 1004 1194 1219 1252]
> Node set dict: {'population': 'All', 'node_id': [115, 145, 215, 355, 378, 568, 1004, 1194, 1219, 1252]}


#### (e) __IDNeuronSet__, based on individual neuron IDs

In [10]:
neuron_set = obi.IDNeuronSet(neuron_ids=(1, 2, 3))
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

IDNeuronSet resolved in population 'All' of circuit 'ToyCircuit-S1-6k':
> Neuron IDs (3): [1 2 3]
> Node set dict: {'population': 'All', 'node_id': [1, 2, 3]}


#### (f) __IDNeuronSet__, based on individual neuron IDs, with random sub-sampling
<u>Note</u>: `random_sample` can be an absolute number or fraction

In [11]:
neuron_set = obi.IDNeuronSet(neuron_ids=range(10), random_sample=0.5, random_seed=999)
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

IDNeuronSet resolved in population 'All' of circuit 'ToyCircuit-S1-6k':
> Neuron IDs (5): [1 3 4 7 9]
> Node set dict: {'population': 'All', 'node_id': [1, 3, 4, 7, 9]}


#### (g) __PropertyNeuronSet__, based on neuron properties
<u>Note</u>: Optionally, instead of keeping the synbolic notation, neuron IDs can be resolved to individual IDs by `force_resolve_ids=True`.

In [12]:
neuron_set = obi.PropertyNeuronSet(property_specs={"layer": [2, 3], "synapse_class": "INH"})
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

# Optional: Individual neuron IDs resolved
print(f"> Node set dict with IDs resolved [OPTIONAL]: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name, force_resolve_ids=True)}")

PropertyNeuronSet resolved in population 'All' of circuit 'ToyCircuit-S1-6k':
> Neuron IDs (200): [ 70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87
  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105
 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123
 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141
 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159
 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177
 178 179 180 181 182 183 184 185 186 187 188 788 789 790 791 792 793 794
 795 796 797 798 799 800 801 802 803 804 805 806 807 808 809 810 811 812
 813 814 815 816 817 818 819 820 821 822 823 824 825 826 827 828 829 830
 831 832 833 834 835 836 837 838 839 840 841 842 843 844 845 846 847 848
 849 850 851 852 853 854 855 856 857 858 859 860 861 862 863 864 865 866
 867 868]
> Node set dict: {'layer': [2, 3], 'synapse_class': 'INH'}
> Node set dict with IDs resol

#### (h) __PropertyNeuronSet__, based on neuron properties, combined with exising (named) node sets
<u>Note</u>: In this case, individual neuron IDs will always be resolved since a combination of properties and node sets is not possible in SONATA node sets otherwise!

In [13]:
neuron_set = obi.PropertyNeuronSet(property_specs={"synapse_class": "INH"}, node_sets=("Layer2", "Layer3"))
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

PropertyNeuronSet resolved in population 'All' of circuit 'ToyCircuit-S1-6k':
> Neuron IDs (200): [ 70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87
  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105
 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123
 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141
 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159
 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177
 178 179 180 181 182 183 184 185 186 187 188 788 789 790 791 792 793 794
 795 796 797 798 799 800 801 802 803 804 805 806 807 808 809 810 811 812
 813 814 815 816 817 818 819 820 821 822 823 824 825 826 827 828 829 830
 831 832 833 834 835 836 837 838 839 840 841 842 843 844 845 846 847 848
 849 850 851 852 853 854 855 856 857 858 859 860 861 862 863 864 865 866
 867 868]
> Node set dict: {'population': 'All', 'node_id': [70, 71, 72, 73, 74, 75, 76, 77, 78, 79

### __Example 3:__ Writing a NeuronSet to a SONATA node set file
<u>Note</u>: A NeuronSet name must be set, which will be the name of the SONATA node set. The name must not exist!

<u>Note 2</u>: The node sets file name is by default taken from the original circuit. An alternative name can optionally be provided.

<u>Note 3</u>: Overwrite (`overwrite_if_exists`) and append (`append_if_exists`) options exist.

In [14]:
output_path = "./"

# Write new file, overwrite if existing
neuron_set = obi.CombinedNeuronSet(name="L123", node_sets=("Layer1", "Layer2", "Layer3"))
nset_file = neuron_set.to_node_set_file(circuit, circuit.default_population_name, output_path=output_path, overwrite_if_exists=True)

# Append to existing file, but name already exists => NOT POSSIBLE
# nset_file = neuron_set.to_node_set_file(circuit, circuit.default_population_name, output_path="./", append_if_exists=True)  # AssertionError: Appending not possible, node set 'Basic' already exists!

# Append to existing file
neuron_set = obi.CombinedNeuronSet(name="L456", node_sets=("Layer4", "Layer5", "Layer6"))
nset_file = neuron_set.to_node_set_file(circuit, circuit.default_population_name, output_path=output_path, append_if_exists=True)

if os.path.exists(nset_file):
    print(f"Node set file: {nset_file}")

Node set file: ./node_sets.json
